In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error


In [2]:
# Batch of inputs (game states, one-hot encoded)
X = tf.placeholder(tf.float32, [None, 13, 26, 9], name="X") # type tf.float32 is needed for the rest of operations

# Batch of outputs (correct predictions of number of actions)
Y_corr = tf.placeholder(tf.float32, [None, 1], name="Y")

In [3]:
# Neural Network Architecture

# Flatten input
X_flat = tf.layers.flatten(X)

# -First Layer - Receives inputs

layer_1 = tf.layers.dense(inputs = X_flat,
                                  units = 512,
                                  activation = tf.nn.relu,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                  use_bias = True,
                                  name="layer_1")

# Dropout probability
dropout_prob = tf.placeholder(tf.float32)

# Add dropout to prevent overfitting
layer_1 = tf.layers.dropout(layer_1, rate=dropout_prob)

with tf.variable_scope("layer_1", reuse=True): # Get weights and bias for histogram
    weights_1 = tf.get_variable("kernel")
    bias_1 = tf.get_variable("bias")
    
# -Second Layer-

layer_2 = tf.layers.dense(inputs = layer_1,
                                  units = 16,
                                  activation = tf.nn.relu,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                  use_bias = True,
                                  name="layer_2")

# Add dropout to prevent overfitting
layer_2 = tf.layers.dropout(layer_2, rate=dropout_prob)

# -Output Layer-

output = tf.layers.dense(inputs = layer_2,
                                  units = 1,
                                  activation = None,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                  use_bias = True,
                                  name="output")

with tf.variable_scope("output", reuse=True): # Get weights and bias for histogram
    weights_output = tf.get_variable("kernel")
    bias_output = tf.get_variable("bias")

In [4]:
# Training
loss = tf.reduce_mean(tf.square(output - Y_corr), name="loss") # Quadratic loss

alfa = tf.placeholder(tf.float64, name='alfa') # Mejor valor alfa=0.001

optimizer = tf.train.AdamOptimizer(learning_rate=alfa, name="optimizer") # Mejor optimizer
train_op = optimizer.minimize(loss, name="train_op")

In [5]:
# TensorBoard

# Summaries
train_loss_sum = tf.summary.scalar('train_loss', loss)

test_loss_sum = tf.summary.scalar('test_loss', loss) # Validation loss

In [6]:
# Session

# Load Training Dataset
dataset = np.load('datasets/dataset_train_3.npz')
dataset_x = dataset['X']
dataset_y = dataset['Y']

# Load Validation Dataset
test_dataset = np.load("datasets/dataset_test.npz") 
test_dataset_x = test_dataset['X']
test_dataset_y = test_dataset['Y']
test_dataset_y_resh = np.reshape(test_dataset_y, (-1, 1)) 

# Number of epochs
num_epochs = 100
# Minibatch size
batch_size = 128 # Best batch size

drop_val = 0.7 # 0.7 and 0.5 best vals
 
# Learning rates to test
dropout_vals = [0] 

for drop_val in dropout_vals:

    with tf.Session() as sess:
        writer = tf.summary.FileWriter("ModeloPlanificacion_log/arch=[512, 16]")
        writer.add_graph(tf.get_default_graph())
        
        sess.run(tf.global_variables_initializer()) # Initialize all variables

        rkf = RepeatedKFold(n_splits=int(len(dataset_y) // batch_size),
                      n_repeats=num_epochs) # Get randomized indexes for minibatches

        # Feed Dict for validation (uses validation set)
        data_dict_test = {X:test_dataset_x, Y_corr:test_dataset_y_resh, dropout_prob:0.0}
        
        # Train the model
        
        it = 0
        
        for _, batch_index in rkf.split(dataset_y):
            batch_x = np.take(dataset_x, batch_index, axis=0) # Obtain current training batch
            batch_y = np.take(dataset_y, batch_index)
            batch_y = np.reshape(batch_y, (-1, 1))
            
            data_dict = {X:batch_x, Y_corr:batch_y, alfa:0.001, dropout_prob:drop_val}
           
            sess.run(train_op, feed_dict=data_dict) # Execute one training step
            
            # Periodically check losses
            if it % 5 == 0:                
                sum2 = sess.run(test_loss_sum, feed_dict=data_dict_test)
                writer.add_summary(sum2, it)
                sum1 = sess.run(train_loss_sum, feed_dict=data_dict)
                writer.add_summary(sum1, it)
        

            it += 1


        # --- After training ---
        
        # Compute MAE
            
        dataset_y_reshaped = np.reshape(dataset_y, (-1, 1))   
        data_dict = {X:dataset_x, dropout_prob:0.0}
        
        y_pred = sess.run(output, feed_dict=data_dict)
        
        mae_base = mean_absolute_error(dataset_y, np.repeat(np.mean(dataset_y), dataset_y.shape[0]))
        mae_model = mean_absolute_error(dataset_y, y_pred)
        
        print("\n\n---MAE TRAIN---\n\n")
        print("Model:", mae_model)
        print("Baseline:", mae_base)
        
        # ----- VALIDATION -----

        # Compute predictions
        data_dict = {X:test_dataset_x, dropout_prob:0.0}
        test_pred = sess.run(output, feed_dict=data_dict)
        
        mae_base_test = mean_absolute_error(test_dataset_y, np.repeat(np.mean(dataset_y), test_dataset_y.shape[0]))
        mae_model_test = mean_absolute_error(test_dataset_y, test_pred)
        
        print("\n\n---MAE VALIDATION--\n\n")
        print("Model:", mae_model_test)
        print("Baseline:", mae_base_test)
    



---MAE TRAIN---


Model: 1.039887220668849
Baseline: 6.7887650132224895


---MAE VALIDATION--


Model: 6.167267234085233
Baseline: 7.099567260502611
